In [ ]:
import os
%cd ..
!pwd

/home/iir/work/ben/NCKU/IIR/SemEval2025_Task9
/home/iir/work/ben/NCKU/IIR/SemEval2025_Task9


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import datetime
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score
import torch.nn.functional as F
from src.MyDataset import MyDataset
from src.BertSentimentClassifier import BertSentimentClassifier
from src.utils import *
import warnings
import wandb
warnings.filterwarnings("ignore")

In [5]:
data = pd.read_csv('data/incidents_train.csv', index_col=0)
project_name  = "ST1_Hazard_Category_V1"
wandb.init(
    project="Thesis_SemEval_2025_Task9",
    name=project_name, 
    config={
        "learning_rate": 2e-5,
        "epochs": 150,
        "batch_size": 400
})

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [6]:
name_bert = "alvaroalon2/biobert_diseases_ner"
tokenizer = BertTokenizer.from_pretrained(name_bert) 

In [12]:
from sklearn.model_selection import train_test_split
import pandas as pd

def stratified_split(csv_path, stratify_column, test_size=0.2, random_state=2024, debug=False):
    """
    Split data into training and development sets using stratified splitting.
    
    Parameters:
    - csv_path (str): Path to the CSV file.
    - stratify_column (str): Column name to use for stratification.
    - test_size (float): Proportion of data to use for the development set (default = 0.2).
    - random_state (int): Seed for random number generator to ensure reproducibility (default = 2024).
    
    Returns:
    - train_df (pd.DataFrame): Training set.
    - dev_df (pd.DataFrame): Development set.
    """
    # Read data from CSV file
    data = pd.read_csv(csv_path, index_col=0)
    
    # Check if the stratify column exists
    if stratify_column not in data.columns:
        raise ValueError(f"Column '{stratify_column}' does not exist in the dataset")

    if debug: 
        # Check distribution of the classes
        print(f"Number of unique classes: {len(data[stratify_column].value_counts())}")
    
    # Perform stratified split
    train_df, dev_df = train_test_split(
        data, 
        test_size=test_size, 
        random_state=random_state, 
        stratify=data[stratify_column]
    )
    
    if debug:
        # Display class distribution in the split sets
        print("\nClass distribution in the training set:")
        print(train_df[stratify_column].value_counts())
        
        print("\nClass distribution in the development set:")
        print(dev_df[stratify_column].value_counts())
    
    return train_df, dev_df


In [13]:
train_df, dev_df = stratified_split(csv_path="data/incidents_train.csv", stratify_column="hazard-category", debug=False)

In [ ]:
label = ""
label_encoder = LabelEncoder()
label_encoder.fit(data["hazard-category"])
train_df['label'] = label_encoder.transform(train_df[label])
dev_df['label'] = label_encoder.transform(dev_df[label])

LabelEncoder()